# Programmatic API Quickstart

Workflow/Manifest/Plugin usage from Python or notebooks. Run from the repo root so sample paths resolve, or use absolute paths.


In [ ]:
import logging
from pathlib import Path

from zyra import plugins
from zyra.manifest import Manifest
from zyra.workflow.api import Workflow

# Load a minimal workflow and inspect stages
wf = Workflow.load(str(Path.cwd() / "samples/workflows/minimal.yml"))
wf.describe()

In [ ]:
# Run with capture + stream to mirror CLI output and still keep buffers
logging.basicConfig(level=logging.INFO)
result = wf.run(capture=True, stream=True)
result.succeeded, result.stages[0].stdout, result.stages[0].stderr

In [ ]:
# Register a local plugin for discovery; this is for manifest/notebook use only (CLI execution not wired)
plugins.register_command("process", "demo_plugin", description="Local demo")
manifest = Manifest.load(stage="process", include_plugins=True)
commands = manifest.list_commands(stage="process")
"process demo_plugin" in commands, commands[:8]

In [ ]:
# Inspect manifest metadata for a specific process command
entries = manifest.describe().get("commands", [])
cmd = next(e for e in entries if e.name.startswith("process "))
cmd.name, cmd.meta

## Notebook sessions and this API

- `zyra.notebook.create_session()` builds stage namespaces from the same manifest used by `Manifest.load()`.
- Plugins registered via `zyra.plugins.register_command` are written to the overlay that notebook sessions read, so they show up in session namespaces and planner/help.
- Workflow APIs stay subprocess-based for parity, while notebook tools call callable wrappers; use whichever fits your notebook flow.

Example:
```python
from zyra.notebook import create_session

sess = create_session()
# Call a process tool directly from the manifest-driven namespace
sess.process.convert_format(file_or_url="/tmp/in.grib2", output="/tmp/out.tif")

# Plugins registered earlier are also discoverable
sess.process.demo_plugin()
```


In [ ]:
from zyra.notebook import create_session

# Notebook session shares the manifest; inline register adds a callable
sess = create_session()
sess.process.register("demo_plugin_session", lambda ns: {"ok": True, "args": vars(ns)})
print("plugin result:", sess.process.demo_plugin_session())
print("available process tools (subset):", sorted(dir(sess.process))[:5])
wrapper = sess.process.convert_format
print("convert-format returns:", wrapper.meta.returns)
wrapper

In [ ]:
# Build a workflow from an in-memory dict (no file needed)
workflow = {
    "name": "Demo Visualization Workflow",
    "stages": [
        {"stage": "acquire", "command": "http", "args": {"url": "https://..."}},
        {"stage": "process", "command": "convert-format", "args": {}},
        {"stage": "visualize", "command": "heatmap", "args": {}},
    ],
}
wf2 = Workflow.from_dict(workflow)
wf2.describe()